In [ ]:
import sys

import numpy as np

from loguru import logger

sys.path.append(r"C:\Users\user\Desktop\github\sotrans_analytics_department")
from source.core.de.serving import Datamart

In [ ]:
logger.remove()
logger.add(sink=sys.stderr, level="TRACE")

# Создание витрины данных

In [ ]:
datamart: Datamart = Datamart(
    agg_cat_cols=("brand_name_1c", "ca_name", "ca_type", "ca_status"),
)

In [ ]:
datamart.IEnd.add_final()

In [ ]:
datamart.df = datamart.df[~datamart.df["brand_name_1c"].isna()]

In [ ]:
for period in (1, 3, 6):
    # Получить сумму продаж за анализируемый период
    datamart.IExpend.add_sale(period=period)
    # Получить количество месяцев с наличием на анализируемый период
    datamart.IHaving.add_stock(period=period)

    for dgt_col in datamart.agg_dgt_cols:
        # Рассчитать ср.знач. продаж с учётом наличия за анализируемый период
        datamart.df.loc[:, f"avg_sale_{dgt_col}_mwh_{period}"] = [
            round(
                np.nan_to_num(
                    np.divide(sales, mwh),
                    nan=0.001,
                    posinf=0.001,
                    neginf=0.001,
                ),
                2,
            )
            for sales, mwh in zip(
                datamart.df[f"ex_sale_cnt_sum_{period}_month"],
                datamart.df[f"hv_stock_{period}_month"],
            )
        ]

        # Рассчитать КТЗ
        datamart.df.loc[:, f"CTS_{dgt_col}_{period}"] = [
            round(
                np.nan_to_num(
                    np.divide(stock, avg_sale),
                    nan=0.001,
                    posinf=0.001,
                    neginf=0.001,
                ),
                2,
            )
            for avg_sale, stock in zip(
                datamart.df[f"avg_sale_{dgt_col}_mwh_{period}"],
                datamart.df[f"en_final_{dgt_col}_fd"],
            )
        ]

In [ ]:
datamart.df = datamart.df.query('ca_type == "дилерский контракт"').query(
    'ca_status == "работаем"'
)

In [ ]:
datamart.df.to_json(
    path_or_buf="coefficient_trade_stock.json", orient="table", index=False
)